<a href="https://colab.research.google.com/github/DynamicLLM/LLM2024/blob/main/src/sample-ai-agent/tradingAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TradingAgents: Multi-Agents LLM Financial Trading Framework

https://tradingagents-ai.github.io/


In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline
from datetime import datetime
import yfinance as yf
import requests

# Cell 1: Define Analyst Agent
class AnalystAgent:
    def __init__(self, symbol):
        self.symbol = symbol

    def get_technical_analysis(self):
        # Fetch historical data
        stock_data = yf.download(self.symbol, period="6mo", interval="1d")
        stock_data['SMA_20'] = stock_data['Close'].rolling(window=20).mean()
        stock_data['RSI'] = self.calculate_rsi(stock_data['Close'])
        return stock_data

    @staticmethod
    def calculate_rsi(series, period=14):
        delta = series.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))


    def get_sentiment_analysis(self, query, start_date, end_date):
        sentiment_analyzer = pipeline("sentiment-analysis")

        # Fetch data from Reddit API
        response = requests.get(
            f"https://api.reddit.com/r/stocks/search.json?q={query}&after={start_date}&before={end_date}",
            headers={"User-Agent": "Mozilla/5.0"}
        )

        try:
            social_data = response.json()
            # Check the structure of the response
            if 'data' in social_data and 'children' in social_data['data']:
                posts = social_data['data']['children']
            else:
                print("Unexpected API response format:", social_data)
                return []

            # Extract titles and analyze sentiment
            sentiments = []
            for post in posts:
                # Safely get 'title' from post data
                title = post.get('data', {}).get('title', None)
                if title:
                    sentiments.append(sentiment_analyzer(title))
                else:
                    print("Post does not contain a title:", post)
            return sentiments

        except Exception as e:
            print("Error processing API response:", e)
            return []



In [2]:
# Cell 2: Define Researcher Agent
class ResearcherAgent:
    def debate(self, technical_data, sentiment_data):
        # Simulate a debate
        bullish = np.random.choice([True, False])  # Simplify debate
        return "BUY" if bullish else "SELL"


In [3]:
# Cell 3: Define Trader Agent
class TraderAgent:
    def make_decision(self, research_insight):
        if research_insight == "BUY":
            return {"action": "BUY", "allocation": 0.1}  # Buy 10% of the portfolio
        elif research_insight == "SELL":
            return {"action": "SELL", "allocation": 0.1}





In [4]:
# Cell 4: Define Risk Management Agent
class RiskManagementAgent:
    def evaluate_risks(self, trader_decision, portfolio):
        if trader_decision["action"] == "BUY" and portfolio["cash"] < trader_decision["allocation"]:
            return "Insufficient Cash"
        return "Approved"



In [5]:
# Cell 5: Define Fund Manager Agent
class FundManagerAgent:
    def execute_trade(self, decision, portfolio):
        allocation_amount = decision["allocation"] * portfolio["cash"]
        if decision["action"] == "BUY":
            portfolio["cash"] -= allocation_amount
            portfolio["stocks"] += allocation_amount / 100  # Simulating stock price of $100/share
        elif decision["action"] == "SELL":
            allocation_amount = decision["allocation"] * portfolio["stocks"]
            portfolio["cash"] += allocation_amount * 100  # Simulating stock price of $100/share
            portfolio["stocks"] -= allocation_amount
        return portfolio


In [6]:
# Cell 6: Main Workflow
# Setup
symbol = "AAPL"
start_date = "2024-01-01"
end_date = "2024-03-31"
portfolio = {"cash": 100000, "stocks": 0}

# Instantiate agents
analyst = AnalystAgent(symbol)
researcher = ResearcherAgent()
trader = TraderAgent()
risk_manager = RiskManagementAgent()
fund_manager = FundManagerAgent()



# Workflow
technical_data = analyst.get_technical_analysis()
sentiment_data = analyst.get_sentiment_analysis(symbol, start_date, end_date)
research_insight = researcher.debate(technical_data, sentiment_data)
trader_decision = trader.make_decision(research_insight)
risk_evaluation = risk_manager.evaluate_risks(trader_decision, portfolio)

print("Technical Analysis:", technical_data.tail())
print("Sentiment Analysis:", sentiment_data)
print("Research Insight:", research_insight)
print("Trader Decision:", trader_decision)
print("Risk Evaluation:", risk_evaluation)

if risk_evaluation == "Approved":
    portfolio = fund_manager.execute_trade(trader_decision, portfolio)

print("Final Portfolio:", portfolio)

[*********************100%***********************]  1 of 1 completed
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Error processing API response: Expecting value: line 1 column 1 (char 0)
Technical Analysis: Price            Close        High         Low        Open    Volume  \
Ticker            AAPL        AAPL        AAPL        AAPL      AAPL   
Date                                                                   
2025-01-03  243.360001  244.179993  241.889999  243.360001  40244100   
2025-01-06  245.000000  247.330002  243.199997  244.309998  45045600   
2025-01-07  242.210007  245.550003  241.350006  242.979996  40856000   
2025-01-08  242.699997  243.710007  240.050003  241.919998  37628900   
2025-01-10  236.850006  240.160004  233.000000  240.009995  61679400   

Price           SMA_20        RSI  
Ticker                             
Date                               
2025-01-03  249.887000  43.879708  
2025-01-06  249.985001  45.992307  
2025-01-07  249.953501  38.659138  
2025-01-08  249.751001  35.424536  
2025-01-10  249.205001  35.026721  
Sentiment Analysis: []
Research Insight: S